**DO NOT EDIT THIS FILE WITHIN THE /TSDS FOLDER - YOU RISK OVERWRITING YOUR WORK THE NEXT TIME YOU PULL FROM THE GITHUB REPOSITORY**

# Assignment 1


### Practical info
* Handin in absalon. The deadline is the 5th of march (see the [course plan](https://github.com/abjer/tsds/wiki/Course-plan))
* You must work in groups of 2-4. **Remember to identify the group members in the filename or in the top of the file contents**.
* If anything is unclear dont hesitate to email me at kuol@econ.ku.dk with questions.

<br>

<br>
<br>
<br>
<br>

# Questions from exercise set 1 (ML recap)
The following questions are drawn from exercise set 1. We have included code that allows you to solve the questions independently of the previous questions in exercise set 1. Note that you might have solved the questions in a different way than we anticipated. In this case the supplied code might need some modification to work with your answer. 

In [ ]:
# Note: there are three .zip files with letter = a,b,c. 
# To ensure the files download in reasonable time we 
# only work with the first of the three. If you have time
# you can modify this cell to download all three. 

import os
import requests

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler        # scales variables to be mean=0,sd=1
from sklearn.linear_model import LogisticRegression     # regression model
from sklearn.pipeline import Pipeline                   # For building our model pipeline
from sklearn.model_selection import GridSearchCV



filenames = []
base_url = 'https://resources.lendingclub.com/'

letter = 'a'
filename = f'LoanStats3{letter}.csv.zip'
url = base_url+filename

if not os.path.exists(filename):
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)
filenames.append(filename)

# Read in csv files, store them
dfs = [pd.read_csv(f,header=0,skiprows=1,low_memory=False) for f in filenames]

# concatenate the dataframes (as standard there is only 1)
df = pd.concat(dfs)\
        .dropna(subset=['loan_amnt'])\
        .dropna(axis=1, how='all')


# Identify loans of interest
df = df.loc[df.loan_status.isin(['Fully Paid', 'Charged Off'])].copy()

# Clean up variables 
df['charged_off'] = (df.loan_status=='Charged Off').astype(int)
df['int_rate_f'] = df.int_rate.str[:-1].astype(float)
df['emp_length_f'] = df.emp_length\
                        .str.split(' ')\
                        .str[0].str[:2]\
                        .str.replace('<','0')\
                        .astype(float)

# label and features
y_var = 'charged_off'
X_vars = ['term', 'int_rate_f', 'grade', 'home_ownership', 'emp_length_f',
          'annual_inc', 'verification_status', 'dti']

# Create dummies
data = pd.get_dummies(df[X_vars+[y_var]], drop_first=True)\
        .dropna()\
        .reset_index(drop=True)\
        .astype(np.float64)\
        .loc[:2000]\
        .copy()


sss = StratifiedShuffleSplit(n_splits=10, test_size=.3, random_state=3)

# These are the row indices of the stratified split
data_splits = list(sss.split(data[y_var], data[y_var]))

# Separate data in y,X
y = data[y_var]
X_vars_b = data.columns!=y_var
X = data.loc[:,X_vars_b]

train_idx, test_idx = data_splits[0]

y_train = y.loc[train_idx]
X_train = X.loc[train_idx]

y_test = y.loc[test_idx]
X_test = X.loc[test_idx]


# Fit vanilla linear model
lr = Pipeline([('scale', StandardScaler()),
               ('clf', LogisticRegression(class_weight='balanced',C=10**10, solver = 'liblinear'))])


lr.fit(X_train, y_train)


# Fit linear model with CV
lr_cv = GridSearchCV(estimator=lr,
                     param_grid={'clf__C':np.logspace(-4,4,5)},
                     n_jobs=-1,
                     cv=3)
lr_cv.fit(X_train, y_train)

> **Ex. 1.1.8:** Apply nested resampling to compute a distribution of test scores with and without optimization. You should use `data_splits` which we defined initially and input all the data.
>
>> *Hint:* You can implement this using your code from Ex. 1.1.6 and combine it with `cross_val_score`. Note that `cv` input should use `data_splits`. See Raschka pp. 188-189 for inspiration. 

In [ ]:
# [Answer to ex.1.1.8 here]

>  **Ex. 1.1.11** Estimate a classification tree on the training data (with default hyperparameters). Evalate both on training and test data by computing the *area under the curve*.
>
>> *Hint:* You can check out code for Ex. 1.1.10 for inspiration. You may also want to look up `roc_auc_score`.

In [ ]:
# [Answer to ex.1.1.11 here]

>  **Ex. 1.1.13** Is Random Forest classification different from the procedure of aggregating tree predictions above? If so, explain how.

In [ ]:
# [Answer to ex.1.1.13 here]

<br>
<br>
<br>
<br>

# Questions from exercise set 2 (ANN 1)
The following questions are drawn from exercise set 2. We have included code that allows you to solve the questions independently of the previous questions in exercise set 1. Note that you might have solved the questions in a different way than we anticipated. In this case the supplied code might need some modification to work with your answer.

In [ ]:
import numpy as np
import random
import matplotlib.pylab as plt
from scipy.interpolate import interp1d


# Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))

def step(z, threshold=0.5):
    if z > threshold:
        return 1
    return 0

# Feed forward neural network class
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        
        # Q: Print these out, explain their contents. You can instantiate a network by
        # doing `net = Network([2, 3, 1])`, and then printing `net.biases`.
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        
        # Q: What is `a`? How many iterations will this loop run? For a `sizes=[2, 3, 1]`
        # network, what is the shape of `a` at each iteration?
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None, silent=False):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        
        n = len(training_data)
        if test_data:
            n_test = len(test_data)
        
        for j in range(epochs):
            
            # Q: What happens here? Why do we shuffle the training data? Explain the
            # contents of `mini_batches`.
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)
            ]
            
            # Q: And what does this step do?
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            
            if not silent:
                if test_data:
                    print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
                else:
                    print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        
        # Q: These two vectors correspond to -∇C(W) (and -∇C(b))
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Q: But what happens here? Explain, in particular, how we update `nabla_b` and `nabla_w`
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        # Q: Now we have our gradient vectors, `nabla_b` and `nabla_w`. Explain how we use them
        # to update the weights and biases
        self.weights = [
            w - eta / len(mini_batch) * nw
            for w, nw in zip(self.weights, nabla_w)
        ]
        self.biases = [
            b - eta / len(mini_batch) * nb
            for b, nb in zip(self.biases, nabla_b)
        ]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book. Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on. It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return nabla_b, nabla_w

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        if self.sizes[-1] == 1:
            test_results = [
                (step(self.feedforward(x)), y)
                for x, y in test_data
            ]
        else:
            test_results = [
                (np.argmax(self.feedforward(x)), y)
                for x, y in test_data
            ]
        return sum(int(y_pred == y) for (y_pred, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return output_activations - y
    
    
# Load in the MNIST data used for ex. 2.2.x    
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

plt.imshow(training_data[0][0].reshape(28, 28))
plt.show()

**Ex. 2.1.2** Using [the dataset](https://playground.tensorflow.org/#activation=sigmoid&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.16631&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false) with the hard-to-seperate circles, create the minimal neural network that seperates the clusters. Again, report your answer with a link.

In [ ]:
# [Answer to ex. 2.1.2]

**Ex. 2.2.4**: Now, fit a model with a suiting architecture (i.e. `sizes`) to `training_data`, and report your accuracy on the `validation_data`.

In [ ]:
# [Answer to ex. 2.2.4]

**Ex. 2.2.5**: Assuming you could get a "pretty" high accuracy in Ex. 2.2.4, Visualize 10 examples that get misclassified (remember to write what the correct label is). comment on what you see.

In [ ]:
# [Answer to ex. 2.2.5]

<br>
<br>
<br>
<br>

# Questions from exercise set 3 (ANN 2)
The following questions are drawn from exercise set 3. 

![img](https://raw.githubusercontent.com/abjer/tsds/master/material_exercises/week_3/2_3_1_net.png)

**Ex. 3.1.2**: Knowing about backpropagation, we actually have everything we need here to compute the gradients of the weights by hand. So go ahead and do that. Report your answer either as a diagram that includes the gradients (you can draw on my figure somehow and insert the resulting image), or just by writing what the gradient of each weight is.
>
> *Hint: When computing gradients with backprop, it can be a bit easier to think of the network as a computational graph. My computational graph looks like [this](https://github.com/abjer/tsds/blob/master/material_exercises/week_3/2_3_1_net_compgraph.png?raw=true).*

In [ ]:
# [Answer to ex. 3.1.2]

<br>
<br>
<br>
<br>

# Questions from exercise set 4 (ANN 3)
The following questions are drawn from exercise set 4. Once again we provide you the code required to answer the question.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import mnist_loader
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable

# Download the data (thanks MIT)
response = requests.get("http://shakespeare.mit.edu/hamlet/full.html")
hamlet = BeautifulSoup(response.content, "html.parser").getText()

# Convert text to character-level one-hot encoding
hamlet_one_hot = pd.get_dummies(pd.Series(list(hamlet)))
character_vec = hamlet_one_hot.columns
x = torch.from_numpy(hamlet_one_hot.values.astype(np.float32))


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        # Neural network layers. Rather than giving the input as a one-hot
        # vector, we represent it as a point in a high-dimensional space (i.e.
        # "an embedding"). This tends to work better.
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input_, hidden):
        signal = self.encoder(input_).view(1, 1, -1)  # Embed input character
        output, hidden = self.rnn(signal, hidden)     # Get output and hidden vector(s)
        prediction = self.decoder(output.view(1, -1))     # Decode to "prediction" vector
        
        return prediction, hidden

    def init_hidden(self):
        return (
            torch.autograd.Variable(torch.zeros(self.n_layers, 1, self.hidden_size)),
            torch.autograd.Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        )
    
epochs = 5000
seq_len = 200
learning_rate = 1e-2
n_layers = 2

model = RNN(len(character_vec), 100, len(character_vec), n_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# We're collecting the losses so we can plot how they (hopefully) decrease over time
all_losses = []
fraction_correct = []
for t in tqdm(range(epochs)):
    
    # Initiate a hidden vector for 
    hidden = model.init_hidden()
    
    # Pick a random input and output sequence. Here we are only taking one sequence
    # per epoch, but normally people take a batch of sequences. Check https://github
    # .com/spro/char-rnn.pytorch for an example on how to do that
    i = np.random.randint(0, x.size(0)-seq_len)
    input_ = torch.max(x[i:i+seq_len], 1)[1]      # torch.max(...)[1] gets the character in terms of its index, so
    target = torch.max(x[i+1:i+1+seq_len], 1)[1]  # input_ will be something like torch.tensor([2, 1, 4, 0, ... 1])
    
    # Backprop through time. Here we are just summing the losses from each timestep
    # and do backpropagation on the variable that holds the sum. PyTorch allows for that
    optimizer.zero_grad()
    loss = 0
    correct = 0
    for j in range(input_.size(0)):
        output, hidden = model(input_[j], hidden)
        loss += loss_fn(output.view(1, -1), target[j].view(1, ))
        correct += int(torch.max(output.view(1, -1), 1)[1][0] == target[j])
    
    # SGD step
    loss.backward()
    optimizer.step()
    
    # Collect loss value for plot
    all_losses.append(float(loss))
    fraction_correct.append(correct / input_.size(0))
        
    # Progress
    if t % 10 == 0:
        print(t, "| loss:", float(loss), "| fraction correct:", fraction_correct[-1])
    
    
plt.plot(all_losses)
plt.show()

> **Ex. 4.2.1**: Train the network for a while (the longer the better) until you feel its error has settled in some local minimum. Then go ahead and generate some gibberish Hamlet with it! To get better results, you can "warm up" the hidden state vectors by first running a sequence of actual Shapespeare through it and then starting generating from the last word in that sequence. Also, what I mean by "start generating" is that instead of predicting output from inputs drawn from your dataset, you input the prediction from the previous timestep and repeat, thus getting something that's completely made up.

In [ ]:
# [Answer to ex. 4.2.1]